<a href="https://colab.research.google.com/github/michael-borck/isys2001-worksheets/blob/main/sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SQL and Python (and a little Pandas)

When it comes to data retrieval and basic statistics, SQL shines, while Python shines for in-depth, flexible exploratory data analysis or data science. What if you could combine the two programming languages into a single application? 

We will use SQLite3 which is part of the standard Python 3 package, so nothing to install. Let’s take a quick look at the data types that are available:

* NULL — Includes a NULL value
* INTEGER — Includes an integer
* REAL — Includes a floating-point (decimal) value
* TEXT. — Includes text
* BLOB. — Includes a binary large object that is stored exactly as input

In this notebook, we’ll demonstrate

* Loading the library
* Creating and connecting to your database
* Creating database tables
* Adding data
* Querying data
* Deleting data



## Import the library

Let’s start off the tutorial by loading in the library. We can do this by using the following command

In [1]:
import sqlite3

## Create a connection

We use the connect method and pass the name of the database. Let's create an *orders.db*.

In [2]:
conn = sqlite3.connect('superheros.db')

We’ve created a new connection object, as well as a new file called orders.db in the directory in which you’re working.


## Create a cursor object

Now that we’ve created a database connection object, our next task is to create a cursor object. A cursor object allows us to execute SQL queries against a database. A cursor acts a middleware between a connection and SQL query

In [3]:
cur = conn.cursor()

# Create Table

Use a common 'pattern'

1. Create a SQL command as a string
2. User cursor to execture the command
3. If needed fetch the results

In [4]:
sql = """
CREATE TABLE IF NOT EXISTS users(
   userid INT PRIMARY KEY,
   fname TEXT,
   lname TEXT,
   gender TEXT);
"""
cur.execute(sql)
conn.commit()

The IF NOT EXISTS will help us when reconnecting to the database. The query will allow us to check if the table exists, and if it does, nothing is changed.

# Insert into Table

Let’s take a look at how to add data with SQLite in Python to the database we just created. Similar to the table generation query, the query to add data uses the cursor object to execute the query.

In [5]:
sql = """
INSERT INTO users(userid, fname, lname, gender) 
   VALUES('00001', 'Nik', 'Piepenbreier', 'male');
"""
cur.execute(sql)
conn.commit()

Often, when we’re working within Python, we’ll have variables that hold values for us. For example, we may have a tuple that contains that information about a user which might look like this:

In [6]:
user = ('00002', 'Lois', 'Lane', 'female')

If we wanted to load this data into our database, we would use a different convention:



In [7]:
cur.execute("INSERT INTO users VALUES(?, ?, ?, ?);", user)
conn.commit()

Incidentally, using the (?, ?, …) method we noted above also helps protect against SQL injection attacks. 

It’s important to note here that the SQLite expects the values to be in tuple-format. However, the variable can contain a list, as long as the list items are tuples. For example, we could add more users using the variable:

In [8]:
more_users = [('00003', 'Peter', 'Parker', 'male'), ('00004', 'Bruce', 'Wayne', 'male'), ('00005', 'Diana', 'Prince', 'female')]

In [9]:
cur.executemany("INSERT INTO users VALUES(?, ?, ?, ?);", more_users)
conn.commit()

In this case, instead of using the execute function, we’ll want to use the executemany function

## Fetching Data

Here we take a look at how to select data with SQLite in Python! We’ll follow a similar structure as we did to execute queries above

In [10]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
one_result = cur.fetchone()
print(one_result)

(1, 'Nik', 'Piepenbreier', 'male')


We wanted to return more than only one result, we could use the fetchmany()

In [11]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
three_results = cur.fetchmany(3)
print(three_results)

[(1, 'Nik', 'Piepenbreier', 'male'), (2, 'Lois', 'Lane', 'Female'), (3, 'Peter', 'Parker', 'Male')]


We canuse the fetchall() function to return all the results.

In [12]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
print(all_results)

[(1, 'Nik', 'Piepenbreier', 'male'), (2, 'Lois', 'Lane', 'Female'), (3, 'Peter', 'Parker', 'Male'), (4, 'Bruce', 'Wayne', 'male'), (5, 'Diana', 'Prince', 'female')]


## Updating Data

## Delete Data

In [13]:
sql='''
DELETE FROM users WHERE lname='Parker';
'''
cur.execute(sql)
conn.commit()

Did it work?

In [14]:
sql='''
select * from users where lname='Parker';
'''
cur.execute(sql)
cur.fetchall()

[]

This prints out an empty list, confirming that the record has been deleted.

## pandas

We can use the pandas package to read a SQLite database

In [15]:
import pandas as pd

sql='''
SELECT * from users;
'''
# Load the data into a DataFrame
users_df = pd.read_sql_query(sql, conn)
users_df

,userid,fname,lname,gender
0,1,Nik,Piepenbreier,male
1,2,Lois,Lane,Female
2,4,Bruce,Wayne,male
3,5,Diana,Prince,female


and create table from a dataframe

In [17]:
# Select only data for females
users_feamle_df = users_df[users_df.gender == 'female']

# Write the new DataFrame to a new SQLite table
users_feamle_df.to_sql("females", conn, if_exists="replace")

## View Schema

Was the new table added.  Lets have a look at the schema.

In [18]:
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',), ('females',)]

Now lets get details on a table

In [21]:
sql='''
PRAGMA table_info('users');
'''
cur.execute(sql)
cur.fetchall()

[(0, 'userid', 'INT', 0, None, 1),
 (1, 'fname', 'TEXT', 0, None, 0),
 (2, 'lname', 'TEXT', 0, None, 0),
 (3, 'gender', 'TEXT', 0, None, 0)]

## Delete (Drop) Table

When the IF EXISTS clause is used along with DROP TABLE syntax SQLite will not report any error message if the table does not exist.

In [22]:
sql='''
DROP TABLE IF EXISTS females;
'''
cur.execute(sql)
cur.fetchall()

[]

Did it work?

In [23]:
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',)]

## Close the connection

The connection must be closed at the end of the session 

In [25]:
conn.close()